In [1]:
import plotly.io as pio
pio.renderers.default = "notebook+pdf"  # Renderer for Notebook and HTML exports + Renderer for PDF exports

ModuleNotFoundError: No module named 'plotly'

# Test

In [ ]:
import io
import pandas as pd
import datetime
import numpy as np
import os

# load dataset
data_path = "/Users/lucie/Desktop/Cours mention/IA/Projet ML/airbnb-berlin-price-prediction-ml-2223/Data/train_airbnb_berlin.csv"
df = pd.read_csv(data_path)

# clean dataset
def clean_df(df):
    to_drop = ['Listing Name', 'Host Name', 'City', 'Country Code', 'Country', 'Square Feet']
    df = df.drop(to_drop, axis = 1)
    df.columns = df.columns.str.replace(' ','_')
    df = df.replace('*', np.nan)
    df = df.replace('%', '', regex = True)

    # set data_type
    data_type = {'Listing_ID' : 'float', 'Host_ID' : 'float', 'Host_Since' : 'datetime64', 'Host_Response_Time' : 'string',
        'Host_Response_Rate' : 'float', 'Is_Superhost' : 'bool', 'neighbourhood' : 'str',
        'Neighborhood_Group' : 'str', 'Postal_Code' : 'float', 'Latitude' : 'float', 'Longitude' : 'float',
        'Is_Exact_Location' : 'bool', 'Property_Type' : 'str', 'Room_Type' : 'str', 'Accomodates' : 'float',
        'Bathrooms' : 'float', 'Bedrooms' : 'float', 'Beds' : 'float', 'Guests_Included' : 'float', 'Min_Nights' : 'float',
        'Reviews' : 'float', 'First_Review' :'datetime64', 'Last_Review' : 'datetime64', 'Overall_Rating' : 'float',
        'Accuracy_Rating' : 'float', 'Cleanliness_Rating' : 'float', 'Checkin_Rating' : 'float',
        'Communication_Rating' : 'float', 'Location_Rating' : 'float', 'Value_Rating' : 'float',
        'Instant_Bookable' : 'bool', 'Business_Travel_Ready' : 'bool', 'Price' : 'float'}

    df = df.astype(data_type)
    return df

df = clean_df(df)

# stats on datas
def print_stats(df): 
    stats_df = pd.DataFrame({
        "min":df.min(numeric_only = True), 
        "max":df.max(numeric_only = True), 
        "mean":df.mean(numeric_only = True),
        "std":df.std(numeric_only = True),
        "median":df.median(numeric_only = True),
        "nunique":df.nunique(), 
        "count_na": df.isna().sum()    
    })
    return stats_df
                             
print_stats(df)

# PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def airbnb_PCA(df, features, target):
    
    # A enlever quand on aura clean la data
    df2 = df.dropna()
    
    # Separating out the features
    x = df2.loc[:, features].values
    # Separating out the target
    y = df2.loc[:,target].values
    # Standardizing the features
    x = StandardScaler().fit_transform(x)
    pca = PCA()
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
    #                           ,columns = ['PC'+str(i) for i in range(1,n+1)]
                              )
    principalDf.index = df2.index
    print(principalDf.shape)
    print(df2[target].shape)
    finalDf = pd.concat([principalDf, pd.Series(df2[target])], axis = 1)
    print(finalDf.shape)
    explained_variance = pca.explained_variance_ratio_
    print(explained_variance)
    
    # Plot the cumulative variance to identify optimal n_components
    fig, ax = plt.subplots()
    xi = np.arange(1, 4, step=1)
    y = np.cumsum(pca.explained_variance_ratio_)
    plt.ylim(0.0,1.1)
    plt.plot(xi, y, marker='o', linestyle='--', color='b')

    plt.xlabel('Number of Components')
    plt.xticks(np.arange(0, 20, step=50)) #change from 0-based array index to 1-based human-readable label
    plt.ylabel('Cumulative variance (%)')
    plt.title('The number of components needed to explain variance')

    plt.axhline(y=0.95, color='r', linestyle='-')
    plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

    ax.grid(axis='x')
    plt.show()
    
    # Return PCA fields and target variable.
    return(finalDf)

In [ ]:
test = airbnb_PCA(df, ['Accomodates', 'Bathrooms', 'Min_Nights'], 'Price')

In [ ]:
df.dropna().shape

In [ ]:
test

In [ ]:
test[test.isnull().any(axis=1)]

# Train / Test / Validation data

In [ ]:
# trucs à faire quand la data sera belle et propre

In [ ]:
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(test, test_size=0.2, random_state=42)


def income_cat_proportions(data):
    return data["Price"].value_counts() / len(data)


compare_props = pd.DataFrame({
    "Input_dataset": income_cat_proportions(test),
    "Test_set": income_cat_proportions(test_set),
}).sort_index()
compare_props["Test set. %error"] = 100 * compare_props["Test_set"] / compare_props["Input_dataset"] - 100

compare_props



In [ ]:
strat_train_set, strat_test_set =\
train_test_split(df, test_size=0.2, stratify=df["Price"], random_state=42, shuffle=True)

compare_props = pd.DataFrame({
    "Input_dataset": income_cat_proportions(df),
    "Test_set": income_cat_proportions(test_set),
    "Stratified": income_cat_proportions(strat_test_set),
}).sort_index()
compare_props["Test set. %error"] = 100 * compare_props["Test_set"] / compare_props["Input_dataset"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Input_dataset"] - 100

compare_props

In [ ]:
# on fait du feature engineering?